## Описание проекта:

**Задача:**

Коммерческий департамент хочет понять, какой тариф приносит больше денег: «Смарт» или «Ультра».<br>

Предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. В распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов, и сделать вывод — какой тариф лучше.





## Общая информация

In [3]:
from scipy import stats as st
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

In [6]:
dt_user = pd.read_csv('users.csv')
dt_calls = pd.read_csv('calls.csv')
dt_messages = pd.read_csv('messages.csv')
dt_internet = pd.read_csv('internet.csv', index_col=0)
dt_tariffs = pd.read_csv('tariffs.csv')

In [7]:
data = [dt_user, dt_calls, dt_messages, dt_internet, dt_tariffs]
for data in data:
    data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   age         500 non-null    int64 
 2   churn_date  38 non-null     object
 3   city        500 non-null    object
 4   first_name  500 non-null    object
 5   last_name   500 non-null    object
 6   reg_date    500 non-null    object
 7   tariff      500 non-null    object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         202607 non-null  object 
 1   call_date  202607 non-null  object 
 2   duration   202607 non-null  float64
 3   user_id    202607 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB
<class 'pandas.core.frame.D

### Вывод:

Файлы читаются, общая информация получена, фатальных ошибок не обнаружено, более детальное изучение ниже.

## Подготовка данных:

### Формирование таблиц

#### Формат даты:

Приведу тип данных в столбцах с датами к формату даты, и так как информация о точной дате мне не потребуется, приведу к формату, удобному для группировки по месяцам: первое число - месяц – год:

In [ ]:
datetime = {'reg_date':dt_user, 'message_date':dt_messages, 'session_date':dt_internet, 'call_date':dt_calls}

for column, data in datetime.items():
    data[column] = pd.to_datetime(data[column],format = '%Y-%m-%d')
    data['mesyc'] = data[column].astype('datetime64[M]')


#### Таблица users (информация о пользователях):

In [ ]:
dt_user.head()

Не вижу смысла заполнять пропущенные значения в churn_date - дата прекращения пользования тарифом, т.к. данные использованы за 2018 год и по крайней мере все пользователи до 31.12.2018 включительно продолжали пользоваться услугами компании.

Приведу тип даных в столбце churn_date формату даты:

In [ ]:
dt_user['churn_date'] = pd.to_datetime(dt_user['churn_date'], format = '%Y-%m-%d')

Добавлю столбец с месяцем прекращения пользования тарифом:

In [ ]:
dt_user['mesyc_churn'] = dt_user['churn_date'].astype('datetime64[M]')

In [ ]:
dt_user.head()

In [ ]:
print('Количество пользователей Мегалайн в выборке:', len(dt_user))

#### Таблица calls (информация о звонках):

Таблица - количество сделанных звонков и израсходованных минут разговора по месяцам для каждого пользователя:

In [ ]:
dt_calls.head()

Посмотрю как распределяется длительность звонков по гистограмме:

In [ ]:
dt_calls['duration'].plot(kind='hist', bins=20)
plt.show()

Очень много значений около нуля. В задании к проекту указано, что каждый звонок округляется отдельно: даже если он длился всего 1 секунду, будет засчитан как 1 минута. Здесь нет необходимости вдаваться в подробности, почему не состоялся звонок, просто я фиксирую факт и значит найду нулевые звонки:

In [ ]:
dt_calls.query('duration == 0.0')

Нулевых звонков нашлось в количестве - 39613. Так как нулевые звонки не принимают участия в исследовании и не входят в выручку, удаляю их:

In [ ]:
dt_calls['duration'] = dt_calls['duration'].replace(0, np.NaN)
dt_calls = dt_calls.dropna()

Проверю:

In [ ]:
dt_calls['duration'].plot(kind='hist', bins=20)
plt.show()

In [ ]:
dt_calls.query('duration == 0.0')

Округлю длительность каждого звонка до минуты вверх и посчитаю длительность разговоров каждого пользователя в месяц. 

In [ ]:
dt_calls['duration'] = np.ceil(dt_calls['duration'])

In [ ]:
dt_calls_mesyc = dt_calls.pivot_table(index = ['user_id', 'mesyc'], values = 'duration', aggfunc = ['count', 'sum'])

In [ ]:
dt_calls_mesyc.head()

#### Таблица messages (информация о сообщениях):

In [ ]:
dt_messages .head()

Таблица - количество отправленных сообщений по месяцам для каждого пользователя:

In [ ]:
dt_mess_mesyc = dt_messages.pivot_table(index = ['user_id', 'mesyc'], values = 'id', aggfunc = 'count')

In [ ]:
dt_mess_mesyc.head()

#### Таблица internet (информация об интернет-сессиях):

In [ ]:
dt_internet.head()

In [ ]:
dt_internet['mb_used'].plot(kind='hist', bins=20)
plt.show()

Очень много значений около нуля. Для веб-трафика отдельные сессии не считаются.  Здесь нет необходимости вдаваться в подробности, почему не состоялось соединение интернет, просто я фиксирую факт и значит найду нулевые звонки:

In [ ]:
dt_internet.query('mb_used == 0.0')

Нулевых интернет-сессий нашлось в количестве - 19598. Так как нулевые интернет-сессии не принимают участия в исследовании и не входят в выручку, удаляю их:

In [ ]:
dt_internet['mb_used'] = dt_internet['mb_used'].replace(0, np.NaN)
dt_internet = dt_internet.dropna()

Проверю:

In [ ]:
dt_internet['mb_used'].plot(kind='hist', bins=20)
plt.show()

In [ ]:
dt_internet.query('mb_used == 0.0')

Таблица - количество израсходованного интернет-трафика по месяцам для каждого пользователя:

In [ ]:
dt_intnt_mesyc = dt_internet.pivot_table(index = ['user_id', 'mesyc'], values = 'mb_used', aggfunc = 'sum')

In [ ]:
dt_intnt_mesyc.head()

#### Сводная таблица: 
- количество сделанных звонков и израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам.

In [ ]:
data_frames = [dt_calls_mesyc, dt_mess_mesyc, dt_intnt_mesyc]

dt_mesyc = reduce(lambda left,right: pd.merge(left,right,on=['user_id', 'mesyc'],
                                            how='outer'), data_frames)
dt_mesyc = dt_mesyc.reset_index()

In [ ]:
dt_mesyc

In [ ]:
dt_prep = pd.merge(dt_mesyc, dt_user.loc[:, ['user_id', 'tariff', 'city' ]], on='user_id')

In [ ]:
new_cols = ['user_id', 'Месяц', 'Звонки', 'Разговоры', 'Сообщения', 'Интернет', 'Тариф', 'Город']
dt_prep.columns = new_cols

В результате объединения получил сводную таблицу dt_prep:

In [ ]:
dt_prep

Проверю на пропуски

In [ ]:
dt_prep.isna().sum()

Заполню пропуски нулевыми значениями:

In [ ]:
dt_prep[['Звонки','Разговоры', 'Сообщения', 'Интернет']] = dt_prep[['Звонки', 'Разговоры', 'Сообщения', 'Интернет']] .fillna(0)

In [ ]:
dt_prep.isna().sum()

In [ ]:
dt_prep.info()

#### Итого:

Общая таблица сформирована dt_prep. Посчитаны для каждого пользователя:<br>

`'Звонки'` - количество сделанных звонков по месяцам;<br> 
`'Разговоры'` - израсходованные минуты разговора по месяцам;<br>
`'Сообщения'` - количество отправленных сообщений по месяцам;<br>
`'Интернет'` - объем израсходованного интернет-трафика по месяцам;


### Расчеты для каждого пользователя

#### Таблица tariffs (информация о тарифах):

In [ ]:
dt_tariffs .head()

#### Добавление данных по тарифу

Добавлю в таблицу dt_prep информацию о тарифах:

In [ ]:
def messages_in(row):
    if row['Тариф'] == 'smart':
        return '50'
    elif row['Тариф'] == 'ultra':
        return '1000'

In [ ]:
dt_prep['messages_included'] = dt_prep.apply(messages_in, axis=1).astype(int)

In [ ]:
def mb_per_month(row):
    if row['Тариф'] == 'smart':
        return '15360'
    elif row['Тариф'] == 'ultra':
        return '30720'

In [ ]:
dt_prep['mb_per_month_included'] = dt_prep.apply(mb_per_month, axis=1).astype(float)

In [ ]:
def minutes_inc(row):
    if row['Тариф'] == 'smart':
        return '500'
    elif row['Тариф'] == 'ultra':
        return '3000'

In [ ]:
dt_prep['minutes_included'] = dt_prep.apply(minutes_inc, axis=1).astype(float)

In [ ]:
def rub_monthly(row):
    if row['Тариф'] == 'smart':
        return '550'
    elif row['Тариф'] == 'ultra':
        return '1950'

In [ ]:
dt_prep['rub_monthly_fee'] = dt_prep.apply(rub_monthly, axis=1).astype(int)

In [ ]:
def rub_per_gb(row):
    if row['Тариф'] == 'smart':
        return '200'
    elif row['Тариф'] == 'ultra':
        return '150'

In [ ]:
dt_prep['rub_per_gb'] = dt_prep.apply(rub_per_gb, axis=1).astype(int)

In [ ]:
def rub_per_message(row):
    if row['Тариф'] == 'smart':
        return '3'
    elif row['Тариф'] == 'ultra':
        return '1'

In [ ]:
dt_prep['rub_per_message'] = dt_prep.apply(rub_per_message, axis=1).astype(int)

In [ ]:
def rub_per_minute(row):
    if row['Тариф'] == 'smart':
        return '3'
    elif row['Тариф'] == 'ultra':
        return '1'

In [ ]:
dt_prep['rub_per_minute'] = dt_prep.apply(rub_per_minute, axis=1).astype(int)

In [ ]:
dt_prep

#### Учёт бесплатного трафика

Добавлю столбец 'трафик_Гб', где сминусую включенный в тариф бесплатные Мб

In [ ]:
dt_prep['трафик_Гб'] = (dt_prep['Интернет'] - dt_prep['mb_per_month_included']) / 1024 # траффик в Гб

In [ ]:
dt_prep

Заменю отрицательные значения в столбце 'трафик_Гб' на нуль:

In [ ]:
dt_prep.loc[(dt_prep['трафик_Гб']<0), 'трафик_Гб'] = 0

In [ ]:
dt_prep['трафик_Гб'] = dt_prep['трафик_Гб'].apply(np.ceil) # округление траффика

In [ ]:
dt_prep

#### Учёт бесплатных минут

Добавлю столбец 'разговоры, мин', где сминусую включенный в тариф бесплатные минуты разговора:

In [ ]:
dt_prep['разговоры, мин'] = dt_prep['Разговоры'] - dt_prep['minutes_included']  

Заменю отрицательные значения в столбце 'разговоры, мин' на нуль:

In [ ]:
dt_prep.loc[(dt_prep['разговоры, мин']<0), 'разговоры, мин'] = 0

In [ ]:
dt_prep

#### Учёт бесплатных сообщений

Добавлю столбец 'разговоры, мин', где сминусую включенный в тариф бесплатные сообщения:

In [ ]:
dt_prep['сообщения'] = dt_prep['Сообщения'] - dt_prep['messages_included']  

Заменю отрицательные значения в столбце 'сообщения' на нуль:

In [ ]:
dt_prep.loc[(dt_prep['сообщения']<0), 'сообщения'] = 0

In [ ]:
dt_prep

#### Помесячная выручка с каждого пользователя

Посчитаю ежемясячную выручку с каждого пользователя с учетом тарифа:

In [ ]:
dt_prep['Выручка'] = (dt_prep['трафик_Гб'] * dt_prep['rub_per_gb']) + (dt_prep['разговоры, мин'] * dt_prep['rub_per_minute']) + (dt_prep['сообщения'] * dt_prep['rub_per_message']) + dt_prep['rub_monthly_fee']

In [ ]:
dt_prep

### Итоговые таблицы

#### Таблица пользователей user:

Создам таблицу user, где выведу столбцы, необходимые для исследования:

In [ ]:
dt_user.head()

In [ ]:
user = dt_user[['user_id', 'city', 'tariff', 'mesyc', 'mesyc_churn']]

In [ ]:
user

Выведу в отдельные столбцы пользователей тарифов Смарт и Ультра с признаками, 1 - пользователь, 0 - нет:

In [ ]:
user.loc[(user["tariff"] == 'smart'), 'smart'] = 1
user.loc[(user["tariff"] == 'ultra'), 'ultra'] = 1

Заполню нулями пропущенные значения и заменю тип данных на int:

In [ ]:
user['smart'] = user['smart'].fillna(0).astype(int)
user['ultra'] = user['ultra'].fillna(0).astype(int)

In [ ]:
user

В таблицу user_reg_churn добавлю информацию об ежемесячном притоке и оттоке пользователей и средннее количество пользователей в месяц для каждого тарифа и в общем: 

In [ ]:
user_reg_ultra = user.pivot_table(index = ['mesyc'], values = 'ultra', aggfunc = 'sum')
user_reg_smart = user.pivot_table(index = ['mesyc'], values = 'smart', aggfunc = 'sum')
user_churn_ultra = user.pivot_table(index = ['mesyc_churn'], values = 'ultra', aggfunc = 'sum')
user_churn_smart = user.pivot_table(index = ['mesyc_churn'], values = 'smart', aggfunc = 'sum')

In [ ]:
user_reg_churn = pd.concat([user_reg_ultra, user_reg_smart, user_churn_ultra, user_churn_smart], axis=1)
user_reg_churn

In [ ]:
user_reg_churn = user_reg_churn.fillna(0).reset_index()
new_colus = ['Месяц','ultra_reg', 'smart_reg', 'ultra_churn', 'smart_churn']
user_reg_churn.columns = new_colus

In [ ]:
user_reg_churn['user_reg'] = (user_reg_churn['ultra_reg'] + user_reg_churn['smart_reg'])
user_reg_churn['user_churn'] = (user_reg_churn['ultra_churn'] + user_reg_churn['smart_churn'])
user_reg_churn['Доля'] = (user_reg_churn['user_churn'] / user_reg_churn['user_reg']).round(2)

In [ ]:
user_reg_churn['ultra_user_sum'] = np.cumsum(user_reg_churn['ultra_reg']) - user_reg_churn['ultra_churn']
user_reg_churn['smart_user_sum'] = np.cumsum(user_reg_churn['smart_reg']) - user_reg_churn['smart_churn']
user_reg_churn['user_sum'] = np.cumsum(user_reg_churn['user_reg']) - user_reg_churn['user_churn']

In [ ]:
user_reg_churn

#### Переменные - количество пользователей тарифов

In [ ]:
nnn_smart = user['smart'].sum()
nnn_ultra = user['ultra'].sum()
print('Количество пользователей тарифа "Смарт":', nnn_smart)
print('Количество пользователей тарифа "Ультра":', nnn_ultra)

#### Общая таблица:

Создам таблицу stat с данными, необходимыми для дальнейшего исследования:

In [ ]:
stat = dt_prep[['user_id', 'Выручка', 'Месяц', 'Звонки', 'Разговоры', 'Сообщения', 'Интернет', 'Тариф', 'Город']] 

In [ ]:
columns = ['Выручка', 'Звонки', 'Разговоры', 'Сообщения', 'Интернет']

for i in columns:
    stat[i] = stat[i].astype(int)

In [ ]:
stat

В таблице ultra_stat отфильтрую данные потребительского характера для тарифа Ультра:

In [ ]:
ultra_stat = stat[(stat['Тариф']=='ultra')]
ultra_stat

В таблице smart_stat отфильтрую данные потребительского характера для тарифа Смарт:

In [ ]:
smart_stat = stat[(stat['Тариф']=='smart')]
smart_stat

In [ ]:
smart_stat['Выручка'].plot(kind='hist', bins=100, figsize=(12,6))
plt.show()

#### Таблицы месячных показателей для тарифов ultra и smart:

В таблице stat_tariff помесячно для каждого тарифа сгруппирую: Выручка, Интернет, Разговоры, Сообщения:

In [ ]:
stat_tariff = stat.pivot_table(index=['Месяц', 'Тариф'],
                    values = ['Разговоры','Сообщения', 'Интернет', 'Выручка'], aggfunc='sum')
stat_tariff = stat_tariff.reset_index()

In [ ]:
stat_tariff

В таблицу ultra сгруппирую помесячные данные по тарифу Ультра для Выручка, Интернет, Разговоры, Сообщения и средние показатели в месяц для одного пользователя в Интернет/user, Разговоры/user, Сообщения/user, Выручка/user:

In [ ]:
ultra = stat_tariff[(stat_tariff['Тариф']=='ultra')]
ultra = pd.merge(ultra, user_reg_churn.loc[:, ['Месяц', 'ultra_user_sum']], on='Месяц')
ultra['Интернет/user'] = (ultra['Интернет'] / ultra['ultra_user_sum']).astype(int)
ultra['Разговоры/user'] = (ultra['Разговоры'] / ultra['ultra_user_sum']).astype(int)
ultra['Сообщения/user'] = (ultra['Сообщения'] / ultra['ultra_user_sum']).astype(int)
ultra['Выручка/user'] = (ultra['Выручка'] / ultra['ultra_user_sum']).astype(int)
ultra

В таблицу smart сгруппирую помесячные данные по тарифу Смарт для Выручка, Интернет, Разговоры, Сообщения и средние показатели в месяц для одного пользователя в Интернет/user, Разговоры/user, Сообщения/user, Выручка/user:

In [ ]:
smart = stat_tariff[(stat_tariff['Тариф']=='smart')]
smart = pd.merge(smart, user_reg_churn.loc[:, ['Месяц', 'smart_user_sum']], on='Месяц')
smart['Интернет/user'] = (smart['Интернет'] / smart['smart_user_sum']).astype(int)
smart['Разговоры/user'] = (smart['Разговоры'] / smart['smart_user_sum']).astype(int)
smart['Сообщения/user'] = (smart['Сообщения'] / smart['smart_user_sum']).astype(int)
smart['Выручка/user'] = (smart['Выручка'] / smart['smart_user_sum']).astype(int)
smart 

#### Группировка пользователей по городу:

Отфильтрую в таблицу moscow, пользователей города Москва:

In [ ]:
moscow = stat[stat['Город'] == 'Москва']
moscow

Отфильтрую в таблицу zamkad, пользователей кроме города Москва:

In [ ]:
zamkad = stat[stat['Город'] != 'Москва']
zamkad

#### Годовые показатели Мегалайн:

In [ ]:
za_god_megalain =stat[['Разговоры','Сообщения', 'Интернет', 'Выручка']].sum()
print(za_god_megalain)

In [ ]:
za_god_ultra = ultra[['Разговоры','Сообщения', 'Интернет', 'Выручка']].sum()
print(za_god_ultra)

In [ ]:
za_god_smart = smart[['Разговоры','Сообщения', 'Интернет', 'Выручка']].sum()
print(za_god_smart)

### вывод

1. Для целей статанализа подготовлены следующие таблицы и переменные:

__nnn_smart__ - Количество пользователей тарифа "Смарт"<br>
__nnn_ultra__ - Количество пользователей тарифа "Ультра"<br>
__stat__ - общая таблица<br>
__ultra_stat__ - общая таблица пользователей тарифа "Ультра"<br>
__smart_stat__ - общая таблица пользователей тарифа "Смарт"<br>
__user__ - таблица пользователей<br>
__user_reg_churn__ - таблица помесячного и потарифного количества пользователей с учетом регистрации и прекращения пользования тарифом<br>
__ultra__ - помесячные показатели и их усредненный значения пользователей тарифа Ультра<br>
__smart__ - помесячные показатели и их усредненный значения пользователей тарифа Смарт<br>
__moscow__ - пользователи г. Москвы<br>
__zamkad__ - пользователи других городов<br>

2. Выводы для статанализа:

Полученные датасеты позволяют при исследовании данных опираться на Закон больших цифр и на центральную предельную теорему, где:

Закон больших чисел — известная закономерность: чем больше раз повторяется эксперимент, тем ближе частота заданного на этом эксперименте события будет к его вероятности. Вероятность события тесно связана с его частотой при многократном повторении эксперимента.

Одна из формулировок центральной предельной теоремы звучит так: если в выборке достаточно наблюдений, выборочное распределение выборочного среднего из любой генеральной совокупности распределено нормально вокруг среднего этой генеральной совокупности. «Любая генеральная совокупность» означает, что сама генеральная совокупность может быть распределена как угодно. Датасет из средних значений выборок всё равно будет нормально распределён вокруг среднего всей
генеральной совокупности. 

Ежемесячное увеличение числа пользователей оператора моделирует увеличение числа наблюдений и число экспериментов. Вероятность события это среднее всей генеральной совокупности. Выборочное распределение выборочного среднего выведены в таблицы __ultra__ и __smart__ , где колонки __nnn/user__ есть средние по каждой выборке. 



## Анализ данных

### Поведение пользователей

#### Оператор - клиенты

In [ ]:
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
user_reg_t = user_reg_churn['user_reg']
user_churn_t = user_reg_churn['user_churn']

x = np.arange(len(labels))  
width = 0.35  

fig, ax = plt.subplots(figsize=(15, 8))
rects1 = ax.bar(x - width/2, user_reg_t, width, label='user_reg')
rects2 = ax.bar(x + width/2, user_churn_t, width, label='user_churn')

ax.set_ylabel('Количество пользователей')
ax.set_xlabel('Месяц')
ax.set_title('Месячный приток и отток клиентов Мегалайн')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
temp_one = user_reg_churn['ultra_user_sum']
temp_two = user_reg_churn['smart_user_sum']

x = np.arange(len(labels))  
width = 0.35  

fig, ax = plt.subplots(figsize=(15, 8))
rects1 = ax.bar(x - width/2, temp_one, width, label='Ultra')
rects2 = ax.bar(x + width/2, temp_two, width, label='Smart')

ax.set_ylabel('Количество пользователей')
ax.set_xlabel('Месяц')
ax.set_title('Помесячный рост клиентской базы с учетом оттока для каждого тарифа')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

#### Описание к Оператор - клиенты:

С августа месяца наблюдается отток клиентов с ежемесячным приростом и снижение ежемесячного количества зарегистрированных клиентов.

Ежемесячно количество клиентов прирастает, значит ежемесячно выборка увеличивается.

#### Тариф - клиенты

In [ ]:
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
temp_one = smart['Интернет/user']
temp_two = ultra['Интернет/user']

x = np.arange(len(labels))  
width = 0.35  

fig, ax = plt.subplots(figsize=(15, 8))
rects1 = ax.bar(x - width/2, temp_one, width, label='smart')
rects2 = ax.bar(x + width/2, temp_two, width, label='ultra')

ax.set_ylabel('Объём трафика')
ax.set_xlabel('Месяц')
ax.set_title('Помесячный объём интернет трафика на одного пользователя для каждого тарифа')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
temp_one = smart['Разговоры/user']
temp_two = ultra['Разговоры/user']

x = np.arange(len(labels))  
width = 0.35  

fig, ax = plt.subplots(figsize=(15, 8))
rects1 = ax.bar(x - width/2, temp_one, width, label='smart')
rects2 = ax.bar(x + width/2, temp_two, width, label='ultra')

ax.set_ylabel('Минут')
ax.set_xlabel('Месяц')
ax.set_title('Помесячное количество минут разговора на одного пользователя для каждого тарифа')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
temp_one = smart['Сообщения/user']
temp_two = ultra['Сообщения/user']

x = np.arange(len(labels))  
width = 0.35  

fig, ax = plt.subplots(figsize=(15, 8))
rects1 = ax.bar(x - width/2, temp_one, width, label='smart')
rects2 = ax.bar(x + width/2, temp_two, width, label='ultra')

ax.set_ylabel('Сообщений')
ax.set_xlabel('Месяц')
ax.set_title('Помесячное количество сообщений на одного полльзователя для каждого тарифа')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

#### Описание к Тариф - клиенты:

Чем больше размер выборки, тем ближе выборочные средние к истинной средней этой совокупности. 

### Статистика

#### Матрица. Распределение.

In [ ]:
pd.plotting.scatter_matrix(stat[['Сообщения', 'Разговоры', 'Интернет']], figsize = (12,12))
plt.show()

In [ ]:
pd.plotting.scatter_matrix(smart_stat[['Сообщения', 'Разговоры', 'Интернет']], figsize = (12,12))
plt.show()

In [ ]:
pd.plotting.scatter_matrix(ultra_stat[['Сообщения', 'Разговоры', 'Интернет']], figsize = (12,12))
plt.show()

Сравнительное распределение плотностей для каждого тарифа и в целом по всей выборке:

In [ ]:
sns.kdeplot(stat['Интернет'])
sns.kdeplot(smart_stat['Интернет'])
sns.kdeplot(ultra_stat['Интернет'])

plt.show()

In [ ]:
sns.kdeplot(stat['Разговоры'])
sns.kdeplot(smart_stat['Разговоры'])
sns.kdeplot(ultra_stat['Разговоры'])
plt.show()

In [ ]:
sns.kdeplot(stat['Сообщения'])
sns.kdeplot(smart_stat['Сообщения'])
sns.kdeplot(ultra_stat['Сообщения'])

plt.show()

##### Описание - Матрица. Распределение.

Здесь наблюдаем классическую форму распределения Пуассона, которая зависит от числа событий. При небольших значениях распределение смещено влево, а с увеличением числа событий становится симметричным и уходит вправо. Очень четко наблюдается на гистограммах.  
Распределение плотности по каждой выборке симметричное и стремится к нормальному. 

#### Показатели статистики

In [ ]:
stat.pivot_table(index = 'Месяц', values = ['Интернет', 'Разговоры', 'Сообщения'], aggfunc = ['mean', 'std', 'var']).round(0)

In [ ]:
smart_stat.pivot_table(index = 'Месяц', values = ['Интернет', 'Разговоры', 'Сообщения'], aggfunc = ['mean', 'std', 'var']).round(0)

In [ ]:
ultra_stat.pivot_table(index = 'Месяц', values = ['Интернет', 'Разговоры', 'Сообщения'], aggfunc = ['mean', 'std', 'var']).round(0)

In [ ]:
stat.mean()

In [ ]:
smart

In [ ]:
smart[['Интернет/user', 'Разговоры/user', 'Сообщения/user', 'Выручка/user']].mean()

In [ ]:
ultra[['Интернет/user', 'Разговоры/user', 'Сообщения/user', 'Выручка/user']].mean()

##### Описание - Показатели статистики

Дисперсия случайной величины - характеристика рассеивания, разбросанности случайной величины около её математического ожидания В нашем случае квадратный корень дисперсии равен приблизительно к стандартному отклонению и близко к фактическому среднему значению. Это говорит о том, что каждая выборка даёт близкое к нормальному распределению.

## Проверка гипотез

Нулевая гипотеза всегда формулируется так, чтобы использовать знак равенства. Другими словами, чтобы все данные лежали в районе изначально предполагаемого значения. Отсюда формулируем гипотезы, требующие проверки в проекте:

- __Гипотеза 1:__

___Нулевая гипотеза___ - средняя выручка пользователей тарифов «Ультра» и «Смарт» равны;

___Альтернативная гипотеза___ - средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются;

- __Гипотеза 2:__  

___Нулевая гипотеза___ - средняя выручка пользователей из Москвы равна выручке пользователей из других регионов

___Альтернативная гипотеза___ - средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов

Параметр equal variance не задаю(по умолчанию = True), потому как выше установили, дисперсии выборок можно считать равными.

alpha = 0.05 - задам стандартное значение. 

### Гипотеза 1

In [ ]:
print('Выручка на одного пользователя тарифа "Смарт" в среднем за год:', smart['Выручка/user'].mean())

In [ ]:
print('Выручка на одного пользователя тарифа "Ультра" в среднем за год:', ultra['Выручка/user'].mean())

In [ ]:
alpha = 0.05
results = st.ttest_ind(smart['Выручка/user'], ultra['Выручка/user'])
print('P-value:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Так как, нулевая гипотеза отвергается, то подтверждается рабочая гипотеза - средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются. И средняя выручка на одного пользователя по каждому тарифу дополнительно подтверждает этот вывод.

### Гипотеза 2

In [ ]:
m = moscow.pivot_table(index = 'user_id', values = 'Выручка', aggfunc = 'sum')
z = zamkad.pivot_table(index = 'user_id', values = 'Выручка', aggfunc = 'sum')

In [ ]:
print('Средняя выручка на одного пользователя в г. Москва', m['Выручка'].mean() / 12)

In [ ]:
print('Средняя выручка на одного пользователя в других городах', z['Выручка'].mean() / 12)

In [ ]:
alpha = 0.05
results = st.ttest_ind(moscow['Выручка'], zamkad['Выручка'])
print('P-value:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Так как, нулевая гипотеза не отвергается, то не подтверждается рабочая гипотеза - средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов. И средняя выручка на одного пользователя по городам дополнительно подтверждает этот вывод.<br>
Здесь не стоит обращать внимания на получившиеся низкие значения средних на каждого пользователя, это усредненная цифра в среднем за год, но так как, я установил, что каждая выборка распределена равномерно вокруг генеральной, то эти значения релевантны.

## Общий вывод

__1. Исходные данные:__<br>
Файлы читаются, общая информация получена, фатальных ошибок не обнаружено, Данные можно считать чистыми.<br>

__2. Теоретическое обоснование:__<br>
К изучению предложена выборка в 500 пользователей за 2018 год, из них:<br> 
Количество пользователей тарифа "Смарт - 351,<br> 
Количество пользователей тарифа "Ультра" – 149.<br>
Количество пользователей из г. Москвы – 99,<br>
Количество пользователей из других городов – 401.<br>

Предложенный объём данных позволяет судить о всей генеральной совокупности по её репрезентативной части.<br> 
Выборка составлена таким образом, что ежемесячно увеличивается размер выборки, то есть получается стратифицированная выборка, более репрезентативная, чем просто случайная.<br> 
Одна из формулировок центральной предельной теоремы звучит так: если в выборке достаточно наблюдений, выборочное распределение выборочного среднего из любой генеральной совокупности распределено нормально вокруг среднего этой генеральной совокупности. «Любая генеральная совокупность» означает, что сама генеральная совокупность может быть распределена, как угодно. Датасет из средних значений выборок всё равно будет нормально распределён вокруг среднего всей генеральной совокупности.<br>
На графиках и гистограммах это отчётливо прослеживается.<br>

__3. Поведение клиентов:__<br>
С августа месяца наблюдается отток клиентов с ежемесячным приростом и снижение ежемесячного количества зарегистрированных клиентов.<br>
Пользователи тарифа Ультра стабильно больше используют ресурсов, против пользователей тарифа Смарт:<br>
- Тариф Ультра:<br>
Интернет/user - 18138 Мб<br>
Разговоры/user – 493 минут<br> 
Сообщения/user – 43 количество<br>

- Тариф Смарт:<br>
Интернет/user - 15056 Мб<br>
Разговоры/user - 386 минут<br>
Сообщения/user - 31 количество<br>

__4. Проверка гипотез.__<br>
Для проверки рабочих гипотез использован метод scipy.stats.ttest_ind() с параметром equal_var = True и стандартным значением alpha = 0.05.<br>
В результате теста:<br>
- рабочая гипотеза - средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются, подтвердилась:<br>
Выручка на одного пользователя тарифа "Смарт" в среднем за год: 1180 рублей,<br>
Выручка на одного пользователя тарифа "Ультра" в среднем за год: 2040 рублей<br>
- рабочая гипотеза - средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов, не подтвердилась. Выручка от одного пользователя приблизительно одинаковы:<br>
Средняя выручка за месяц на одного среднесписочного пользователя г. Москва: 795 рублей,<br>
Средняя выручка за месяц на одного среднесписочного пользователя в других городах 829 рублей.<br>

__5. Рекомендация коммерческому департаменту:__<br>
Выручка Мегалайн по выборке за год составила 4914451 рублей.<br> 
Из них на тариф Ультра приходится - 2039100 рублей<br>
и на тариф Смарт - 2875351 рублей.<br>
То есть, в целом тариф Смарт вносит в общий бюджет больше денег нежели тариф Ультра. Но при этом, надо учитывать, что каждый пользователь тарифа Ультра в среднем приносит 2040 рублей, а каждый пользователь тарифа Смарт - 1180 рублей.

__Исходя из показателей выручки, корректировка рекламного бюджета в пользу тарифа Ультра смотрится предпочтительнее.__